In [1]:
#env 로드
import os
import topogenesis as tg
import trimesh as tm
import numpy as np
import pyvista as pv
import panel as pn
import pandas as pd 
import copy
import csv

import ipywidgets as widgets
from ladybug.sunpath import Sunpath #ladybug sunpath 호출
from IPython.display import display, Image
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import MinMaxScaler

# csv에서 lattice 배열 만드는 함수 정의
def lattice_from_csv(file_path):

    meta_df = pd.read_csv(file_path, nrows=3)

    shape = np.array(meta_df['shape'])       # 시작 좌표 [x_min, y_min, z_min] 예를 들어, [-55000, -7000, 0] x축 -55000, y축 -7000, z축 0의 위치
    unit = np.array(meta_df['unit'])         # 몇개의 셀로 구성되는지 [x_shape, y_shape, z_shape] 예를 들어 [56, 35, 24]이면 x축 56개, y 35개, z 24개의 셀을 가집니다.
    minbound = np.array(meta_df['minbound']) # 각 셀의 크기. 예를 들어, [1000, 1000, 3000] 각 셀의 크기는 x축 1000 단위, y축 1000 단위, z축 3000 단위

    lattice_df = pd.read_csv(file_path, skiprows=5) # 앞의 5개 줄을 스킵하고 6개 부터 데이터를 불러오기
    buffer = np.array(lattice_df['value']).reshape(shape)
    l = tg.to_lattice(buffer, minbound=minbound, unit=unit)
    
    return l

# tri_to_pv 함수 정의: 삼각형 메시(tri_mesh)를 PyVista 형식으로 변환합니다.
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0), (1, 0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)    
    return pv_mesh

# 1이라는 범위에 대한 이웃패턴 정의 (중심을 포함함)
stencil = tg.create_stencil("von_neumann", 1, 1)
stencil.set_index([0,0,0], 0)


In [2]:
#csv 로드
csv_path = os.path.relpath('original_lattice/interior_lattice.csv')
#obj 포인트 csv 로드
context_mesh = tm.load(os.path.relpath('obj\context.obj'))  # 주변환경 obj 메시 불러오기

envelope_lattice = tg.lattice_from_csv(csv_path)
avail_lattice = tg.lattice_from_csv(csv_path)                                         # CSV 파일에서 격자 데이터 불러오기
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)             # 복사된 데이터를 격자 객체로 변환

In [3]:
df = pd.read_excel('excel/program (이노션).xlsx', sheet_name='Sheet1')
maxfloors = df[['max_floor']].copy()
program_prefs = df[['sun_acc', 'ent_acc', 'str_acc', 'ung_pre', 'dist_facade','top_pre']].copy()

program_prefs.reset_index(drop=True, inplace=True)
fields = {}  # 필드를 저장할 빈 딕셔너리를 생성

for f in program_prefs.columns:  # 각 필드에 대한 파일 경로 설정    
    lattice_path = os.path.relpath('matrixs/' + f + '.csv')      
    try:        
        fields[f] = tg.lattice_from_csv(lattice_path)  # 배열 파일에서 데이터를 불러와서 fields 딕셔너리에 저장
    except:        
        fields[f] = copy.deepcopy(avail_lattice * 0 + 1)# 파일이 없을 경우, 모든 값을 1로 설정한 기본 격자 데이터를 fields 딕셔너리에 저장

top_pre_index = df.columns.get_loc('top_pre') # 'top_pre' 열의 인덱스 찾기 (5번째에 위치)
program_mtx = df.iloc[:, top_pre_index + 1:]  # 'top_pre' 열 이후 모든 열을 포함하는 새로운 데이터프레임 생성 (프로그램 상관관계 matrix 찾기)
program_mtx.columns = range(len(program_mtx.columns)) # 새로운 데이터프레임의 열 이름을 0부터 시작하는 숫자로 변경     (프로그램 번호입력 0~ ) 

space_list = df['space_name'].to_dict() # 'space_name' 열을 딕셔너리로 변환하여 'space_list'에 저장
sizes_complete = df['vox_amount']# 'vox_amount' 열의 값을 가져와 'sizes_complete'에 저장

agent_areas = [] # 'agent_areas' 리스트 초기화

for area in sizes_complete: # 'sizes_complete'의 각 값에 대해 반복 
    agent_areas.append(round(int(area)))    # 각 값을 정수로 변환하고 반올림하여 'agent_areas'에 추가

occ_lattice = avail_lattice * 0 - 1

print (agent_areas)
print(program_mtx)
print(maxfloors)

[500, 250, 333, 50, 5000, 300, 100, 50, 40, 20, 20, 60, 500, 100, 200, 100, 8, 15, 20, 30, 45, 50, 10, 50, 20, 50, 30, 40, 30, 60, 100, 300, 116, 150, 50, 20, 20, 40, 20, 300, 200, 200, 100, 150, 100, 20, 30, 50, 30, 40, 50]
     0    1    2    3    4    5    6    7    8    9   ...   41   42   43   44  \
0   1.0  0.9  1.0  0.7  0.7  0.1  0.1  0.3  0.6  0.6  ...  0.7  0.5  0.6  0.1   
1   0.9  1.0  0.1  1.0  0.3  0.1  0.1  0.1  0.6  0.5  ...  0.1  0.6  0.1  0.1   
2   1.0  0.1  1.0  0.3  1.0  0.1  0.1  0.3  0.5  0.1  ...  0.1  0.1  0.1  0.1   
3   0.7  1.0  0.3  1.0  0.1  1.0  1.0  1.0  0.1  0.1  ...  0.3  0.4  0.6  0.5   
4   0.7  0.3  1.0  0.1  1.0  1.0  1.0  0.5  0.1  0.1  ...  0.4  0.6  0.6  0.2   
5   0.1  0.1  0.1  1.0  1.0  1.0  0.5  1.0  0.1  0.1  ...  0.4  0.6  0.2  0.1   
6   0.1  0.1  0.1  1.0  1.0  0.5  1.0  0.1  1.0  1.0  ...  0.4  0.1  0.1  0.1   
7   0.3  0.1  0.3  1.0  0.5  1.0  0.1  1.0  1.0  1.0  ...  0.1  0.1  0.1  0.1   
8   0.6  0.6  0.5  0.1  0.1  0.1  1.0  1.0  1.

In [4]:
agn_csv_path = os.path.relpath('original_lattice/starting_locs.csv')
agn_locs = []

with open(agn_csv_path, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # 헤더 행을 건너뜁니다
    for row in csv_reader:
        agent_id = int(row[0])
        location = tuple(map(int, row[1:]))
        agn_locs.append((agent_id, location))

print(agn_locs)

[(0, (39, 8, 5)), (1, (36, 11, 5)), (2, (36, 10, 5)), (3, (33, 14, 6)), (4, (35, 18, 7)), (5, (30, 18, 18)), (6, (35, 18, 8)), (7, (29, 18, 23)), (8, (28, 18, 18)), (9, (28, 18, 22)), (10, (29, 18, 18)), (11, (30, 17, 21)), (12, (27, 19, 21)), (13, (28, 19, 20)), (14, (28, 19, 21)), (15, (26, 18, 21)), (16, (26, 17, 21)), (17, (26, 18, 22)), (18, (26, 17, 22)), (19, (27, 18, 21)), (20, (26, 18, 20)), (21, (31, 19, 9)), (22, (31, 19, 8)), (23, (28, 17, 23)), (24, (35, 18, 12)), (25, (30, 18, 16)), (26, (32, 18, 9)), (27, (32, 18, 10)), (28, (33, 18, 10)), (29, (33, 18, 9)), (30, (35, 18, 5)), (31, (34, 18, 7)), (32, (36, 18, 5)), (33, (34, 18, 5)), (34, (28, 18, 0)), (35, (28, 19, 16)), (36, (38, 18, 0)), (37, (32, 18, 0)), (38, (35, 18, 6)), (39, (16, 18, 7)), (40, (31, 19, 21)), (41, (33, 18, 20)), (42, (26, 16, 20)), (43, (27, 20, 19)), (44, (30, 18, 20)), (45, (15, 18, 6)), (46, (23, 18, 0)), (47, (26, 19, 0)), (48, (33, 18, 0)), (49, (35, 19, 5)), (50, (18, 19, 0))]


In [5]:
import numpy as np
from scipy import ndimage
import pandas as pd

def initialize_lattice(avail_lattice, agn_locs):
    occ_lattice = np.full(avail_lattice.shape, -1, dtype=int)
    for agent_id, position in agn_locs:
        occ_lattice[position] = agent_id
    return occ_lattice

def distance(a_id, fns, lattice_cens, agn_locs):
    fns_cens = lattice_cens[np.ravel_multi_index(np.array(fns).T, avail_lattice.shape)]
    agn_avg_loc = np.average(agn_locs[a_id], axis=0)
    dist_m = np.sqrt(((fns_cens - agn_avg_loc)**2).sum(axis=1))
    return dist_m


In [6]:

# 주어진 스텐실을 기반으로 이웃을 캐싱하는 함수
def cache_neighbors(lattice, stencil):
    neighbors_cache = {}
    for x in range(lattice.shape[0]):  # x축을 따라 루프
        for y in range(lattice.shape[1]):  # y축을 따라 루프
            for z in range(lattice.shape[2]):  # z축을 따라 루프
                loc = (x, y, z)  # 현재 위치를 (x, y, z)로 지정
                # 주어진 스텐실을 사용하여 현재 위치에서 이웃을 찾고 캐시에 저장
                neighbors_cache[loc] = lattice.find_neighbours_masked(stencil, loc=loc, id_type="3D")
    return neighbors_cache

# 스텐실 정의
stencil = tg.create_stencil("von_neumann", 1, 1)
stencil_sq = tg.create_stencil("von_neumann", 1, 1)
stencil_sq.set_index([0, 0, 0], 0)  # 중심 인덱스 비활성화
stencil_sq.set_index([0, 0, 1], 0)  # z축 방향으로 비활성화
stencil_sq.set_index([0, 0, -1], 0)  # z축 방향으로 비활성화

# 이웃 캐시 생성
neighbors_cache = cache_neighbors(avail_lattice, stencil)
neighbors_sq_cache = cache_neighbors(avail_lattice, stencil_sq)

In [7]:
# # 시뮬레이션 루프 - 지정된 프레임 수만큼 반복
# while t < n_frames:  
#     n_fns = 0  # 빈 이웃이 없는 에이전트의 수를 세는 변수 초기화

#     # 각 에이전트의 선호도에 대해 반복
#     for a_id, a_prefs in program_prefs.iterrows():  
#         # agn_locs[a_id]가 리스트가 아닌 경우 리스트로 변환
#         if isinstance(agn_locs[a_id], (tuple, int)):
#             agn_locs[a_id] = [agn_locs[a_id]]

#         # 현재 에이전트의 위치를 가져옴
#         a_locs = agn_locs[a_id]
#         # a_locs가 int형이면 이를 격자 인덱스로 변환      

#         free_neighs = []  # 빈 이웃 위치를 저장할 리스트 초기화

#         # 에이전트의 각 위치에 대한 루프
#         for a_loc in a_locs:
#             # 격자 크기가 유효한 경우
#             if avail_lattice.shape[0] > 0 and avail_lattice.shape[1] > 0 and avail_lattice.shape[2] > 0:  
#                 neighs = neighbors_cache[tuple(a_loc)]  # 현재 위치의 이웃을 가져옴
#             else:
#                 print("avail_lattice shape is invalid:", avail_lattice.shape)
#                 neighs = []  # 유효하지 않으면 빈 리스트로 설정
                
#             # 빈 이웃 목록에 추가
#             for n in neighs:
#                 if avail_lattice[tuple(n)]:  # 이웃이 비어 있으면
#                     free_neighs.append(tuple(n))  # 튜플로 변환하여 추가

#         # 빈 이웃이 없는 경우
#         if not free_neighs:  
#             n_fns += 1  # 빈 이웃이 없는 에이전트 수 증가

#         # 빈 이웃이 있는 경우
#         if free_neighs:  
#             fns = np.array(free_neighs)  # 빈 이웃을 배열로 변환

#             # 외부 평가: 각 빈 이웃에 대한 평가 값 계산
#             a_eval = np.sum([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)

#             # 각 프로그램에 대한 거리와 가중치 평가
#             for s in program_mtx.columns:
#                 s = int(s)
#                 vals = distance(s, fns)  # 거리 값을 계산
#                 a_weighted_vals = vals * program_mtx.loc[a_id, s]  # 가중치 적용
#                 a_eval += a_weighted_vals  # 평가 값에 추가

#             free_neighs_set = set(free_neighs)  # 빈 이웃을 집합으로 변환

#                         # 비율 평가 함수 적용
#             for i, n in enumerate(free_neighs):
#                 ratio_score = evaluate_ratio(a_locs, n, target_ratio)
#                 a_eval[i] *= ratio_score  # 비율 점수가 낮을수록 선호되도록 설정

#             # 수평 이동을 선호하도록 가중치 적용
#             horizontal_bias = 3 # 수평 방향으로의 가중치 설정
#             vertical_penalty = 0.1  # 수직 방향 이동에 대한 패널티 설정
#             for i, n in enumerate(free_neighs):
#                 if abs(n[1] - a_locs[0][1]) > 0:  # y축 방향으로 이동
#                     a_eval[i] *= horizontal_bias
#                 if abs(n[2] - a_locs[0][2]) > 0:  # z축 방향으로 이동
#                     a_eval[i] *= vertical_penalty      


#             current_length = len(a_locs)  # 현재 에이전트가 점유한 공간의 길이
#             i_eval = np.zeros(current_length)  # 내부 선택을 위한 평가 값 초기화
#             max_space = np.rint(agent_areas[a_id]).astype(int)  # 최대 공간 크기 계산

#             # 현재 길이가 최대 공간 이상인 경우
#             if current_length >= max_space:  
#                 i_eval = np.ones(current_length)  # 내부 평가 값 초기화

#                 # 내부 평가: 현재 위치의 평가 값 계산
#                 for f in program_prefs.columns:
#                     vals = fields[f][np.array(a_locs)[:, 0], np.array(a_locs)[:, 1], np.array(a_locs)[:, 2]]
#                     a_weighted_vals = vals * a_prefs[f]
#                     i_eval *= a_weighted_vals

#                 for s in program_mtx.columns:
#                     s = int(s)
#                     vals = distance(s, np.array(a_locs))
#                     a_weighted_vals = vals * program_mtx.loc[a_id, s]
#                     i_eval *= a_weighted_vals

#                 i_neighs_count = np.zeros(current_length)

#                 # 이웃 평가: 현재 위치의 이웃 수 계산
#                 for id, a_loc in enumerate(a_locs):
#                     neighs = neighbors_cache[tuple(a_loc)]
#                     for n in neighs:
#                         if avail_lattice[tuple(n)] and (occ_lattice[tuple(n)] == a_id):
#                             i_neighs_count[id] += 1

#             # 내부 선택 인덱스 계산 (i_eval이 최소인 인덱스 선택)
#             selected_int_inner = np.argmin(i_eval)
#             # 외부 선택 인덱스 계산 (a_eval이 최대인 인덱스 선택)
#             selected_int = np.argmax(a_eval)

#             # 내부 평가가 외부 평가보다 낮은 경우
#             if current_length >= max_space and i_eval[selected_int_inner] < a_eval[selected_int]:  
#                 selected_inner_3d_id = tuple(a_locs[selected_int_inner])
#                 a_locs.pop(selected_int_inner)  # 내부 위치 제거
#                 avail_lattice[selected_inner_3d_id] = True
#                 occ_lattice[selected_inner_3d_id] = -1

#             # 공간이 꽉 차지 않은 경우
#             if current_length < max_space:  
#                 selected_neigh_3d_id = tuple(free_neighs[selected_int])
#                 a_locs.append(selected_neigh_3d_id)  # 빈 이웃에 추가
#                 avail_lattice[selected_neigh_3d_id] = False
#                 occ_lattice[selected_neigh_3d_id] = a_id

#         agn_locs[a_id] = a_locs  # 업데이트된 위치 저장

#     new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)  # 새로운 격자 생성
#     frames.append(new_occ_lattice)  # 프레임에 추가
#     print(t, "/", n_fns, end="  ")  # 현재 프레임과 빈 이웃이 없는 에이전트 수 출력
#     t += 1  # 시간 증가


In [8]:
# def simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, max_spaces, 
#                     fields, program_prefs, program_mtx):
#     frames = []
    
#     for t in range(n_frames):
#         print(f"Frame {t+1}/{n_frames}")
#         for a_id, a_prefs in program_prefs.iterrows():
#             a_locs = agents[a_id]
            
#             # 사용 가능한 모든 이웃 찾기
#             all_neighbors = set()
#             for a_loc in a_locs:
#                 neighs = neighbors_cache[a_loc]
#                 for n in neighs:
#                     if avail_lattice[tuple(n)] and occ_lattice[tuple(n)] == -1:
#                         all_neighbors.add(tuple(n))
            
#             if all_neighbors:
#                 fns = np.array(list(all_neighbors))
                
#                 # 평가 점수 계산
#                 a_eval = np.sum([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)
                
#                 # 거리 및 방향성 평가
#                 center = np.mean(a_locs, axis=0)
#                 directions = fns - center
#                 distances = np.linalg.norm(directions, axis=1)
                
#                 # 수평 방향 선호
#                 horizontal_score = np.abs(directions[:, 0]) + np.abs(directions[:, 1])
#                 vertical_penalty = np.abs(directions[:, 2])

#                 a_eval += horizontal_score * 5.0 - vertical_penalty * 0.5
#                 a_eval /= (distances + 1)  # 거리에 따른 패널티
                
#                 # 비율 평가
#                 for i, n in enumerate(fns):
#                     ratio_score = evaluate_ratio(a_locs, n, target_ratio)
#                     a_eval[i] *= ratio_score
                
#                 # 최고 점수의 이웃 선택
#                 sorted_indices = np.argsort(a_eval)[::-1]
#                 cells_to_add = min(5, max_spaces[a_id] - len(a_locs), len(all_neighbors))
                
#                 for idx in sorted_indices[:cells_to_add]:
#                     new_cell = tuple(fns[idx])
#                     a_locs.append(new_cell)
#                     avail_lattice[new_cell] = False
#                     occ_lattice[new_cell] = a_id
            
#             agents[a_id] = a_locs
        
#         frames.append(np.copy(occ_lattice))
    
#     return frames, agents

# # 시뮬레이션 실행
# n_frames = 100
# agents = {agent_id: [tuple(position)] for agent_id, position in agn_locs}
# max_spaces = {agent_id: space for agent_id, space in enumerate(agent_areas)}

# frames, final_agents = simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, max_spaces, 
#                                        fields, program_prefs, program_mtx)

In [9]:
# # 시뮬레이션 루프 - 지정된 프레임 수만큼 반복
# while t < n_frames:  
#     n_fns = 0  # 빈 이웃이 없는 에이전트의 수를 세는 변수 초기화

#     # 각 에이전트의 선호도에 대해 반복
#     for a_id, a_prefs in program_prefs.iterrows():  
#         # agn_locs[a_id]가 리스트가 아닌 경우 리스트로 변환
#         if isinstance(agn_locs[a_id], (tuple, int)):
#             agn_locs[a_id] = [agn_locs[a_id]]

#         # 현재 에이전트의 위치를 가져옴
#         a_locs = agn_locs[a_id]
#         # a_locs가 int형이면 이를 격자 인덱스로 변환      

#         free_neighs = []  # 빈 이웃 위치를 저장할 리스트 초기화

#         # 에이전트의 각 위치에 대한 루프
#         for a_loc in a_locs:
#             # 격자 크기가 유효한 경우
#             if avail_lattice.shape[0] > 0 and avail_lattice.shape[1] > 0 and avail_lattice.shape[2] > 0:  
#                 neighs = neighbors_cache[tuple(a_loc)]  # 현재 위치의 이웃을 가져옴
#             else:
#                 print("avail_lattice shape is invalid:", avail_lattice.shape)
#                 neighs = []  # 유효하지 않으면 빈 리스트로 설정
                
#             # 빈 이웃 목록에 추가
#             for n in neighs:
#                 if avail_lattice[tuple(n)]:  # 이웃이 비어 있으면
#                     free_neighs.append(tuple(n))  # 튜플로 변환하여 추가

#         # 빈 이웃이 없는 경우
#         if not free_neighs:  
#             n_fns += 1  # 빈 이웃이 없는 에이전트 수 증가

#         # 빈 이웃이 있는 경우
#         if free_neighs:  
#             fns = np.array(free_neighs)  # 빈 이웃을 배열로 변환

#             # 외부 평가: 각 빈 이웃에 대한 평가 값 계산
#             a_eval = np.sum([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)

#             # 각 프로그램에 대한 거리와 가중치 평가
#             for s in program_mtx.columns:
#                 s = int(s)
#                 vals = distance(s, fns)  # 거리 값을 계산
#                 a_weighted_vals = vals * program_mtx.loc[a_id, s]  # 가중치 적용
#                 a_eval += a_weighted_vals  # 평가 값에 추가

#             free_neighs_set = set(free_neighs)  # 빈 이웃을 집합으로 변환

#                         # 비율 평가 함수 적용
#             for i, n in enumerate(free_neighs):
#                 ratio_score = evaluate_ratio(a_locs, n, target_ratio)
#                 a_eval[i] *= ratio_score  # 비율 점수가 낮을수록 선호되도록 설정

#             # 수평 이동을 선호하도록 가중치 적용
#             horizontal_bias = 3 # 수평 방향으로의 가중치 설정
#             vertical_penalty = 0.1  # 수직 방향 이동에 대한 패널티 설정
#             for i, n in enumerate(free_neighs):
#                 if abs(n[1] - a_locs[0][1]) > 0:  # y축 방향으로 이동
#                     a_eval[i] *= horizontal_bias
#                 if abs(n[2] - a_locs[0][2]) > 0:  # z축 방향으로 이동
#                     a_eval[i] *= vertical_penalty      


#             current_length = len(a_locs)  # 현재 에이전트가 점유한 공간의 길이
#             i_eval = np.zeros(current_length)  # 내부 선택을 위한 평가 값 초기화
#             max_space = np.rint(agent_areas[a_id]).astype(int)  # 최대 공간 크기 계산

#             # 현재 길이가 최대 공간 이상인 경우
#             if current_length >= max_space:  
#                 i_eval = np.ones(current_length)  # 내부 평가 값 초기화

#                 # 내부 평가: 현재 위치의 평가 값 계산
#                 for f in program_prefs.columns:
#                     vals = fields[f][np.array(a_locs)[:, 0], np.array(a_locs)[:, 1], np.array(a_locs)[:, 2]]
#                     a_weighted_vals = vals * a_prefs[f]
#                     i_eval *= a_weighted_vals

#                 for s in program_mtx.columns:
#                     s = int(s)
#                     vals = distance(s, np.array(a_locs))
#                     a_weighted_vals = vals * program_mtx.loc[a_id, s]
#                     i_eval *= a_weighted_vals

#                 i_neighs_count = np.zeros(current_length)

#                 # 이웃 평가: 현재 위치의 이웃 수 계산
#                 for id, a_loc in enumerate(a_locs):
#                     neighs = neighbors_cache[tuple(a_loc)]
#                     for n in neighs:
#                         if avail_lattice[tuple(n)] and (occ_lattice[tuple(n)] == a_id):
#                             i_neighs_count[id] += 1

#             # 내부 선택 인덱스 계산 (i_eval이 최소인 인덱스 선택)
#             selected_int_inner = np.argmin(i_eval)
#             # 외부 선택 인덱스 계산 (a_eval이 최대인 인덱스 선택)
#             selected_int = np.argmax(a_eval)

#             # 내부 평가가 외부 평가보다 낮은 경우
#             if current_length >= max_space and i_eval[selected_int_inner] < a_eval[selected_int]:  
#                 selected_inner_3d_id = tuple(a_locs[selected_int_inner])
#                 a_locs.pop(selected_int_inner)  # 내부 위치 제거
#                 avail_lattice[selected_inner_3d_id] = True
#                 occ_lattice[selected_inner_3d_id] = -1

#             # 공간이 꽉 차지 않은 경우
#             if current_length < max_space:  
#                 selected_neigh_3d_id = tuple(free_neighs[selected_int])
#                 a_locs.append(selected_neigh_3d_id)  # 빈 이웃에 추가
#                 avail_lattice[selected_neigh_3d_id] = False
#                 occ_lattice[selected_neigh_3d_id] = a_id

#         agn_locs[a_id] = a_locs  # 업데이트된 위치 저장

#     new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)  # 새로운 격자 생성
#     frames.append(new_occ_lattice)  # 프레임에 추가
#     print(t, "/", n_fns, end="  ")  # 현재 프레임과 빈 이웃이 없는 에이전트 수 출력
#     t += 1  # 시간 증가


In [10]:
# def simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, max_spaces, 
#                     fields, program_prefs, program_mtx):
#     frames = []
#     max_vertical_growth = 3  # 최대 수직 성장 제한
    
#     for t in range(n_frames):
#         print(f"Frame {t+1}/{n_frames}")
#         for a_id, a_prefs in program_prefs.iterrows():
#             a_locs = agents[a_id]
            
#             # 현재 에이전트의 최소/최대 높이 계산
#             current_min_z = min(loc[2] for loc in a_locs)
#             current_max_z = max(loc[2] for loc in a_locs)
            
#             # 사용 가능한 모든 이웃 찾기
#             all_neighbors = set()
#             for a_loc in a_locs:
#                 neighs = neighbors_cache[a_loc]
#                 for n in neighs:
#                     n = tuple(n)
#                     if (avail_lattice[n] and occ_lattice[n] == -1 and 
#                         current_min_z - 1 <= n[2] <= current_max_z + 1):  # 수직 성장 제한
#                         all_neighbors.add(n)
            
#             if all_neighbors:
#                 fns = np.array(list(all_neighbors))
                
#                 # 평가 점수 계산
#                 a_eval = np.sum([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)
                
#                 # 거리 및 방향성 평가
#                 center = np.mean(a_locs, axis=0)
#                 directions = fns - center
#                 distances = np.linalg.norm(directions, axis=1)
                
#                 # 수평 방향 강력 선호, 수직 방향 강력 페널티
#                 horizontal_score = np.abs(directions[:, 0]) + np.abs(directions[:, 1])
#                 vertical_penalty = np.abs(directions[:, 2]) ** 2  # 제곱하여 페널티 강화

#                 a_eval += horizontal_score * 2.0 - vertical_penalty * 1.0
#                 a_eval /= (distances + 1)  # 거리에 따른 패널티
                
#                 # 비율 평가
#                 for i, n in enumerate(fns):
#                     ratio_score = evaluate_ratio(a_locs, n, target_ratio)
#                     a_eval[i] *= ratio_score
                
#                 # 최고 점수의 이웃 선택
#                 sorted_indices = np.argsort(a_eval)[::-1]
#                 cells_to_add = min(5, max_spaces[a_id] - len(a_locs), len(all_neighbors))
                
#                 added_cells = 0
#                 for idx in sorted_indices:
#                     if added_cells >= cells_to_add:
#                         break
#                     new_cell = tuple(fns[idx])
#                     if current_min_z <= new_cell[2] <= current_max_z + 1:  # 수직 성장 추가 제한
#                         a_locs.append(new_cell)
#                         avail_lattice[new_cell] = False
#                         occ_lattice[new_cell] = a_id
#                         added_cells += 1
            
#             agents[a_id] = a_locs
        
#         frames.append(np.copy(occ_lattice))
    
#     return frames, agents

# # 시뮬레이션 실행
# n_frames = 50
# agents = {agent_id: [tuple(position)] for agent_id, position in agn_locs}
# max_spaces = {agent_id: space for agent_id, space in enumerate(agent_areas)}

# frames, final_agents = simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, max_spaces, 
#                                        fields, program_prefs, program_mtx)

In [11]:
# def simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, neighbors_sq_cache, max_spaces, 
#                     fields, program_prefs, program_mtx):
#     frames = []
#     max_vertical_growth = 3  # 최대 수직 성장 제한 (층 수)
    
#     for t in range(n_frames):
#         print(f"Frame {t+1}/{n_frames}")
#         for a_id, a_prefs in program_prefs.iterrows():
#             if a_id not in agents:
#                 print(f"Warning: Agent {a_id} not found in agents dictionary. Skipping.")
#                 continue
            
#             a_locs = agents[a_id]
            
#             if not a_locs or not all(isinstance(loc, tuple) and len(loc) == 3 for loc in a_locs):
#                 print(f"Warning: Agent {a_id} has invalid location data. Skipping.")
#                 continue
            
#             if len(a_locs) >= max_spaces[a_id]:
#                 continue  # 최대 공간에 도달한 에이전트는 건너뜁니다
            
#             # 현재 에이전트의 범위 계산
#             current_min_z = min(loc[2] for loc in a_locs)
#             current_max_z = max(loc[2] for loc in a_locs)
            
#             # 사용 가능한 모든 이웃 찾기 (수직 및 수평 이웃 포함)
#             all_neighbors = set()
#             for a_loc in a_locs:
#                 sq_neighs = neighbors_sq_cache[a_loc]  # 수평 이웃
#                 all_neighs = neighbors_cache[a_loc]  # 모든 이웃 (수직 포함)
                
#                 for n in sq_neighs:
#                     n = tuple(n)
#                     if current_min_z <= n[2] <= current_max_z:
#                         if avail_lattice[n]:
#                             all_neighbors.add((n, 0.7))  # 수평 이웃에 0.7 가중치 부여
                
#                 for n in all_neighs:
#                     n = tuple(n)
#                     if current_min_z <= n[2] <= min(current_max_z + 1, current_min_z + max_vertical_growth - 1):
#                         if avail_lattice[n]:
#                             if (n, 0.7) not in all_neighbors:  # 수평 이웃이 아닌 경우에만 추가
#                                 all_neighbors.add((n, 0.3))  # 수직 이웃에 0.3 가중치 부여
            
#             if all_neighbors:
#                 fns, weights = zip(*all_neighbors)
#                 fns = np.array(fns)
#                 weights = np.array(weights)
                
#                 # 평가 점수 계산
#                 a_eval = np.sum([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)
                
#                 # 거리 및 방향성 평가
#                 center = np.mean(a_locs, axis=0)
#                 directions = fns - center
#                 distances = np.linalg.norm(directions, axis=1)
                
#                 # 수평 및 수직 방향 평가
#                 horizontal_score = np.abs(directions[:, 0]) + np.abs(directions[:, 1])
#                 vertical_score = np.abs(directions[:, 2])

#                 # 가중치 적용
#                 a_eval += (horizontal_score * 0.7 + vertical_score * 0.3) * weights
#                 a_eval /= (distances + 1)  # 거리에 따른 패널티
                
#                 # Squareness 평가
#                 current_shape = np.max(a_locs, axis=0) - np.min(a_locs, axis=0)
#                 for i, n in enumerate(fns):
#                     new_shape = np.maximum(np.max(a_locs, axis=0), n) - np.minimum(np.min(a_locs, axis=0), n)
#                     shape_score = 1 / (1 + np.std(new_shape[:2]))  # x와 y 방향의 표준편차가 작을수록 높은 점수
#                     a_eval[i] *= shape_score
                
#                 # 최고 점수의 이웃 선택
#                 sorted_indices = np.argsort(a_eval)[::-1]
#                 cells_to_add = min(5, max_spaces[a_id] - len(a_locs), len(all_neighbors))
                
#                 added_cells = 0
#                 for idx in sorted_indices:
#                     if added_cells >= cells_to_add:
#                         break
#                     new_cell = tuple(fns[idx])
#                     if avail_lattice[new_cell]:
#                         a_locs.append(new_cell)
#                         avail_lattice[new_cell] = False
#                         occ_lattice[new_cell] = a_id
#                         added_cells += 1
            
#             agents[a_id] = a_locs
        
#         frames.append(np.copy(occ_lattice))
    
#     return frames, agents

# # 시뮬레이션 실행
# n_frames = 500
# agents = {agent_id: [position] for agent_id, position in agn_locs}
# max_spaces = {agent_id: space for agent_id, space in enumerate(agent_areas)}

# frames, final_agents = simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, neighbors_sq_cache, max_spaces, 
#                                        fields, program_prefs, program_mtx)

In [12]:

# def simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, neighbors_sq_cache, max_spaces, 
#                     fields, program_prefs, program_mtx, target_ratio):
#     frames = []
#     max_vertical_growth = 5  # 최대 수직 성장 제한 (층 수)
    
#     for t in range(n_frames):
#         print(f"Frame {t+1}/{n_frames}")
#         for a_id, a_prefs in program_prefs.iterrows():
#             a_locs = agents[a_id]
            
#             if len(a_locs) >= max_spaces[a_id]:
#                 continue  # 최대 공간에 도달한 에이전트는 건너뜁니다
            
#             # 현재 에이전트의 범위 계산
#             current_min_z = min(loc[2] for loc in a_locs)
#             current_max_z = max(loc[2] for loc in a_locs)
            
#             # 수평 이웃 찾기
#             horizontal_neighbors = set()
#             for a_loc in a_locs:
#                 neighs = neighbors_sq_cache[a_loc]
#                 for n in neighs:
#                     n = tuple(n)
#                     if n[2] == a_loc[2]:  # 같은 층의 이웃만 고려
#                         if avail_lattice[n] and occ_lattice[n] == -1:
#                             horizontal_neighbors.add(n)
            
#             # 수평 공간이 가득 찼는지 확인
#             horizontal_full = len(horizontal_neighbors) == 0
            
#             # 수직 이웃 찾기
#             vertical_neighbors = set()
#             if horizontal_full:
#                 for a_loc in a_locs:
#                     if a_loc[2] < current_min_z + max_vertical_growth - 1:
#                         up_neighbor = (a_loc[0], a_loc[1], a_loc[2] + 1)
#                         if up_neighbor[2] < occ_lattice.shape[2] and avail_lattice[up_neighbor] and occ_lattice[up_neighbor] == -1:
#                             vertical_neighbors.add(up_neighbor)
            
#             all_neighbors = horizontal_neighbors.union(vertical_neighbors)
            
#             if all_neighbors:
#                 fns = np.array(list(all_neighbors))
                
#                 # 평가 점수 계산
#                 a_eval = np.sum([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)
                
#                 # 거리 및 방향성 평가
#                 center = np.mean(a_locs, axis=0)
#                 directions = fns - center
#                 distances = np.linalg.norm(directions, axis=1)
                
#                 # 수평 및 수직 방향 평가
#                 horizontal_score = np.abs(directions[:, 0]) + np.abs(directions[:, 1])
#                 vertical_score = np.abs(directions[:, 2])

#                 # 수직 이동에 대한 보너스 (수평 공간이 가득 찼을 때)
#                 vertical_bonus = 2.0 if horizontal_full else 0.2
                
#                 # 비율 평가
#                 ratio_scores = np.array([evaluate_ratio(a_locs, tuple(n), target_ratio) for n in fns])
                
#                 # 최종 점수 계산
#                 a_eval += horizontal_score * 0.8 + vertical_score * vertical_bonus
#                 a_eval *= ratio_scores * 10  # 비율 점수에 높은 가중치 부여
#                 a_eval /= (distances + 1)  # 거리에 따른 패널티
                
#                 # 최고 점수의 이웃 선택
#                 sorted_indices = np.argsort(a_eval)[::-1]
#                 cells_to_add = min(10, max_spaces[a_id] - len(a_locs), len(all_neighbors))
                
#                 for idx in sorted_indices[:cells_to_add]:
#                     new_cell = tuple(fns[idx])
#                     a_locs.append(new_cell)
#                     avail_lattice[new_cell] = False
#                     occ_lattice[new_cell] = a_id
#                     if len(a_locs) >= max_spaces[a_id]:
#                         break
            
#             agents[a_id] = a_locs
        
#         frames.append(np.copy(occ_lattice))
    
#     return frames, agents

# # 시뮬레이션 실행
# n_frames = 200
# agents = {agent_id: [tuple(position)] for agent_id, position in agn_locs}
# max_spaces = {agent_id: space for agent_id, space in enumerate(agent_areas)}
# target_ratio = (3, 3, 1)  # 예시 비율, 필요에 따라 조정

# frames, final_agents = simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, neighbors_sq_cache, max_spaces, 
#                                        fields, program_prefs, program_mtx, target_ratio)

In [13]:
# def simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, neighbors_sq_cache, max_spaces, 
#                     fields, program_prefs, program_mtx):
#     frames = []
#     max_vertical_growth = 3  # 최대 수직 성장 제한 (층 수)
    
#     for t in range(n_frames):
#         print(f"Frame {t+1}/{n_frames}")
#         for a_id, a_prefs in program_prefs.iterrows():
#             a_locs = agents[a_id]
            
#             if len(a_locs) >= max_spaces[a_id]:
#                 continue  # 최대 공간에 도달한 에이전트는 건너뜁니다
            
#             # 현재 에이전트의 범위 계산
#             current_min_z = min(loc[2] for loc in a_locs)
#             current_max_z = max(loc[2] for loc in a_locs)
            
#             # 사용 가능한 모든 이웃 찾기 (수직 제한 포함)
#             all_neighbors = set()
#             for a_loc in a_locs:
#                 neighs = neighbors_sq_cache[a_loc]  # 수평 이웃만 고려
#                 for n in neighs:
#                     n = tuple(n)
#                     if current_min_z <= n[2] <= min(current_max_z + 1, current_min_z + max_vertical_growth - 1):
#                         if avail_lattice[n] or (occ_lattice[n] != -1 and occ_lattice[n] != a_id):
#                             all_neighbors.add(n)
            
#             if all_neighbors:
#                 fns = np.array(list(all_neighbors))
                
#                 # 평가 점수 계산
#                 a_eval = np.sum([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] for f in program_prefs.columns], axis=0)
                
#                 # 거리 및 방향성 평가
#                 center = np.mean(a_locs, axis=0)
#                 directions = fns - center
#                 distances = np.linalg.norm(directions, axis=1)
                
#                 # 수평 및 수직 방향 평가
#                 horizontal_score = np.abs(directions[:, 0]) + np.abs(directions[:, 1])
#                 vertical_score = np.abs(directions[:, 2])

#                 # 수직 이동 선호 확률 (예: 20%)
#                 vertical_preference = np.random.choice([1, -1], size=len(fns), p=[0.2, 0.8])
                
#                 # 수직 이동에 대한 페널티 감소 및 선호도 반영
#                 a_eval += horizontal_score * 0.8 + vertical_score * vertical_preference * 0.2
#                 a_eval /= (distances + 1)  # 거리에 따른 패널티
                
#                 # Squareness 평가
#                 current_shape = np.max(a_locs, axis=0) - np.min(a_locs, axis=0)
#                 for i, n in enumerate(fns):
#                     new_shape = np.maximum(np.max(a_locs, axis=0), n) - np.minimum(np.min(a_locs, axis=0), n)
#                     shape_score = 1 / (1 + np.std(new_shape[:2]))  # x와 y 방향의 표준편차가 작을수록 높은 점수
#                     a_eval[i] *= shape_score
                
#                 # 최고 점수의 이웃 선택
#                 sorted_indices = np.argsort(a_eval)[::-1]
#                 cells_to_add = min(10, max_spaces[a_id] - len(a_locs), len(all_neighbors))
                
#                 for idx in sorted_indices[:cells_to_add]:
#                     new_cell = tuple(fns[idx])
#                     if occ_lattice[new_cell] != -1 and occ_lattice[new_cell] != a_id:
#                         # 다른 에이전트의 공간을 차지하려는 경우
#                         other_id = occ_lattice[new_cell]
#                         other_score = np.sum([fields[f][new_cell] * program_prefs.loc[other_id, f] for f in program_prefs.columns])
#                         if a_eval[idx] > other_score * 1.5:  # 20% 이상 높은 점수일 때만 차지
#                             agents[other_id].remove(new_cell)
#                         else:
#                             continue  # 점수가 충분히 높지 않으면 이 셀은 건너뜁니다
                    
#                     a_locs.append(new_cell)
#                     avail_lattice[new_cell] = False
#                     occ_lattice[new_cell] = a_id
            
#             agents[a_id] = a_locs
        
#         frames.append(np.copy(occ_lattice))
    
#     return frames, agents

# # 시뮬레이션 실행
# n_frames = 200
# agents = {agent_id: [tuple(position)] for agent_id, position in agn_locs}
# max_spaces = {agent_id: space for agent_id, space in enumerate(agent_areas)}

# frames, final_agents = simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, neighbors_sq_cache, max_spaces, 
#                                        fields, program_prefs, program_mtx)

                                

In [14]:
import numpy as np

def calculate_aspect_ratio(locs):
    if len(locs) < 2:
        return 1.0
    mins = np.min(locs, axis=0)
    maxs = np.max(locs, axis=0)
    dims = maxs - mins + 1
    return max(dims[0] / dims[1], dims[1] / dims[0])

def simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, neighbors_sq_cache, max_spaces, 
                    fields, program_prefs, program_mtx):
    frames = []
    max_vertical_growth = 7 # 최대 수직 성장 제한 (층 수)
    max_aspect_ratio = 2.0  # 최대 허용 가로세로 비율
    
    # occ_lattice를 numpy 배열로 초기화
    occ_lattice = np.full(avail_lattice.shape, -1, dtype=int)
    for agent_id, locs in agents.items():
        for loc in locs:
            occ_lattice[loc] = agent_id
    
    for t in range(n_frames):
        print(f"Frame {t+1}/{n_frames}")
        for a_id, a_prefs in program_prefs.iterrows():
            a_locs = agents[a_id]
            
            if len(a_locs) >= max_spaces[a_id]:
                continue  # 최대 공간에 도달한 에이전트는 건너뜁니다
            
            # 현재 에이전트의 범위 계산
            current_min_z = min(loc[2] for loc in a_locs)
            current_max_z = max(loc[2] for loc in a_locs)
            
            # 수평 이웃 찾기
            horizontal_neighbors = set()
            for a_loc in a_locs:
                neighs = neighbors_sq_cache[a_loc]
                for n in neighs:
                    n = tuple(n)
                    if n[2] == a_loc[2]:  # 같은 층의 이웃만 고려
                        if avail_lattice[n] and occ_lattice[n] == -1:
                            horizontal_neighbors.add(n)
            
            # 수평 공간이 가득 찼는지 확인
            horizontal_full = len(horizontal_neighbors) == 0
            
            # 수직 이웃 찾기
            vertical_neighbors = set()
            if horizontal_full:
                for a_loc in a_locs:
                    if a_loc[2] < current_min_z + max_vertical_growth - 1:
                        up_neighbor = (a_loc[0], a_loc[1], a_loc[2] + 1)
                        if up_neighbor[2] < occ_lattice.shape[2] and avail_lattice[up_neighbor] and occ_lattice[up_neighbor] == -1:
                            vertical_neighbors.add(up_neighbor)
            
            all_neighbors = horizontal_neighbors.union(vertical_neighbors)
            
            if all_neighbors:
                fns = np.array(list(all_neighbors))
                
                # 프로그램 선호도에 따른 평가 점수 계산
                a_eval = np.zeros(len(fns))
                for f in program_prefs.columns:
                    a_eval += fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] * 10  # 가중치 증가

                # 프로그램 간 관계 평가
                for other_id in agents.keys():
                    if other_id != a_id:
                        other_locs = np.array(agents[other_id])
                        if len(other_locs) > 0:
                            distances = np.min(np.linalg.norm(fns[:, np.newaxis] - other_locs, axis=2), axis=1)
                            relation_score = program_mtx.loc[a_id, other_id] / (distances + 1)
                            a_eval += relation_score * 5  # 관계 점수의 가중치 조정
                
                # 거리 및 방향성 평가
                center = np.mean(a_locs, axis=0)
                directions = fns - center
                distances = np.linalg.norm(directions, axis=1)
                
                # 수평 및 수직 방향 평가
                horizontal_score = np.abs(directions[:, 0]) + np.abs(directions[:, 1])
                vertical_score = np.abs(directions[:, 2])

                # 수직 이동에 대한 보너스 (수평 공간이 가득 찼을 때)
                vertical_bonus = 2.0 if horizontal_full else 0.5
                
                a_eval += horizontal_score * 0.8 + vertical_score * vertical_bonus
                a_eval /= (distances + 1)  # 거리에 따른 패널티
                
                # 비율 평가
                for i, n in enumerate(fns):
                    new_locs = a_locs + [tuple(n)]
                    aspect_ratio = calculate_aspect_ratio(new_locs)
                    if aspect_ratio > max_aspect_ratio:
                        a_eval[i] *= (max_aspect_ratio / aspect_ratio) ** 2
                
                # 최고 점수의 이웃 선택
                sorted_indices = np.argsort(a_eval)[::-1]
                cells_to_add = min(10, max_spaces[a_id] - len(a_locs), len(all_neighbors))
                
                for idx in sorted_indices[:cells_to_add]:
                    new_cell = tuple(fns[idx])
                    new_locs = a_locs + [new_cell]
                    if calculate_aspect_ratio(new_locs) <= max_aspect_ratio:
                        a_locs.append(new_cell)
                        avail_lattice[new_cell] = False
                        occ_lattice[new_cell] = a_id
                    if len(a_locs) >= max_spaces[a_id]:
                        break
            
            agents[a_id] = a_locs
        
        frames.append(np.copy(occ_lattice))

        
    return frames, occ_lattice

# 시뮬레이션 실행
n_frames = 1000
agents = {agent_id: [tuple(position)] for agent_id, position in agn_locs}
max_spaces = {agent_id: space for agent_id, space in enumerate(agent_areas)}

frames, final_agents = simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, neighbors_sq_cache, max_spaces, 
                                       fields, program_prefs, program_mtx)



                                    

Frame 1/1000
Frame 2/1000
Frame 3/1000
Frame 4/1000
Frame 5/1000
Frame 6/1000
Frame 7/1000
Frame 8/1000
Frame 9/1000
Frame 10/1000
Frame 11/1000
Frame 12/1000
Frame 13/1000
Frame 14/1000
Frame 15/1000
Frame 16/1000
Frame 17/1000
Frame 18/1000
Frame 19/1000
Frame 20/1000
Frame 21/1000
Frame 22/1000
Frame 23/1000
Frame 24/1000
Frame 25/1000
Frame 26/1000
Frame 27/1000
Frame 28/1000
Frame 29/1000
Frame 30/1000
Frame 31/1000
Frame 32/1000
Frame 33/1000
Frame 34/1000
Frame 35/1000
Frame 36/1000
Frame 37/1000
Frame 38/1000
Frame 39/1000
Frame 40/1000
Frame 41/1000
Frame 42/1000
Frame 43/1000
Frame 44/1000
Frame 45/1000
Frame 46/1000
Frame 47/1000
Frame 48/1000
Frame 49/1000
Frame 50/1000
Frame 51/1000
Frame 52/1000
Frame 53/1000
Frame 54/1000
Frame 55/1000
Frame 56/1000
Frame 57/1000
Frame 58/1000
Frame 59/1000
Frame 60/1000
Frame 61/1000
Frame 62/1000
Frame 63/1000
Frame 64/1000
Frame 65/1000
Frame 66/1000
Frame 67/1000
Frame 68/1000
Frame 69/1000
Frame 70/1000
Frame 71/1000
Frame 72/1000
F

In [15]:
# def calculate_aspect_ratio(locs):
#     if len(locs) < 2:
#         return 1.0
#     mins = np.min(locs, axis=0)
#     maxs = np.max(locs, axis=0)
#     dims = maxs - mins + 1
#     return max(dims[0] / dims[1], dims[1] / dims[0])

# def simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, neighbors_sq_cache, max_spaces, 
#                     fields, program_prefs, program_mtx):
#     frames = []
#     max_vertical_growth = 7 # 최대 수직 성장 제한 (층 수)
#     max_aspect_ratio = 3.0  # 최대 허용 가로세로 비율
    
#     # occ_lattice를 numpy 배열로 초기화
#     occ_lattice = np.full(avail_lattice.shape, -1, dtype=int)
#     for agent_id, locs in agents.items():
#         for loc in locs:
#             occ_lattice[loc] = agent_id
    
#     for t in range(n_frames):
#         print(f"Frame {t+1}/{n_frames}")
#         # 에이전트 순서를 랜덤하게 섞습니다
#         agent_order = list(program_prefs.index)
#         np.random.shuffle(agent_order)
        
#         for a_id in agent_order:
#             a_prefs = program_prefs.loc[a_id]
#             a_locs = agents[a_id]
            
#             if len(a_locs) >= max_spaces[a_id]:
#                 continue  # 최대 공간에 도달한 에이전트는 건너뜁니다
            
#             # 현재 에이전트의 범위 계산
#             current_min_z = min(loc[2] for loc in a_locs)
#             current_max_z = max(loc[2] for loc in a_locs)
      
#             # 모든 이웃 찾기 (수평 + 수직)
#             all_neighbors = set()
#             for a_loc in a_locs:
#                 neighs = neighbors_sq_cache[a_loc]
#                 for n in neighs:
#                     n = tuple(n)
#                     if current_min_z <= n[2] <= min(current_max_z + 1, current_min_z + max_vertical_growth - 1):
#                         if avail_lattice[n]:
#                             all_neighbors.add(n)
            
#             if all_neighbors:
#                 fns = np.array(list(all_neighbors))
                
#                 # 프로그램 선호도에 따른 평가 점수 계산
#                 a_eval = np.zeros(len(fns))
#                 for f in program_prefs.columns:
#                     a_eval += fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] * a_prefs[f] * 10  # 가중치 증가

#                 # 프로그램 간 관계 평가
#                 for other_id in agents.keys():
#                     if other_id != a_id:
#                         other_locs = np.array(agents[other_id])
#                         if len(other_locs) > 0:
#                             distances = np.min(np.linalg.norm(fns[:, np.newaxis] - other_locs, axis=2), axis=1)
#                             relation_score = program_mtx.loc[a_id, other_id] / (distances + 1)
#                             a_eval += relation_score * 5  # 관계 점수의 가중치 조정
                
#                 # 거리 및 방향성 평가
#                 center = np.mean(a_locs, axis=0)
#                 directions = fns - center
#                 distances = np.linalg.norm(directions, axis=1)
                
#                 # 수평 및 수직 방향 평가
#                 horizontal_score = np.abs(directions[:, 0]) + np.abs(directions[:, 1])
#                 vertical_score = np.abs(directions[:, 2])

#                 a_eval += horizontal_score * 0.8 + vertical_score * 0.2
#                 a_eval /= (distances + 1)  # 거리에 따른 패널티
                
#                 # 비율 평가
#                 for i, n in enumerate(fns):
#                     new_locs = a_locs + [tuple(n)]
#                     aspect_ratio = calculate_aspect_ratio(new_locs)
#                     if aspect_ratio > max_aspect_ratio:
#                         a_eval[i] *= (max_aspect_ratio / aspect_ratio) ** 2
                
#                 # 최고 점수의 이웃 선택
#                 sorted_indices = np.argsort(a_eval)[::-1]
#                 cells_to_add = min(10, max_spaces[a_id] - len(a_locs), len(all_neighbors))
                
#                 for idx in sorted_indices[:cells_to_add]:
#                     new_cell = tuple(fns[idx])
#                     new_locs = a_locs + [new_cell]
#                     if calculate_aspect_ratio(new_locs) <= max_aspect_ratio:
#                         current_occupant = occ_lattice[new_cell]
#                         if current_occupant == -1 or a_eval[idx] > calculate_cell_score(new_cell, current_occupant, program_prefs, fields) * 1.2:
#                             if current_occupant != -1:
#                                 agents[current_occupant].remove(new_cell)
#                             a_locs.append(new_cell)
#                             avail_lattice[new_cell] = False
#                             occ_lattice[new_cell] = a_id
#                     if len(a_locs) >= max_spaces[a_id]:
#                         break
            
#             agents[a_id] = a_locs
        
#         frames.append(np.copy(occ_lattice))
        
#     return frames, occ_lattice

# def calculate_cell_score(cell, agent_id, program_prefs, fields):
#     score = 0
#     for f in program_prefs.columns:
#         score += fields[f][cell] * program_prefs.loc[agent_id, f]
#     return score

# # 시뮬레이션 실행
# n_frames = 100
# agents = {agent_id: [tuple(position)] for agent_id, position in agn_locs}
# max_spaces = {agent_id: space for agent_id, space in enumerate(agent_areas)}

# frames, final_agents = simulate_growth(n_frames, agents, avail_lattice, occ_lattice, neighbors_cache, neighbors_sq_cache, max_spaces, 
#                                        fields, program_prefs, program_mtx)

In [16]:
from matplotlib.colors import ListedColormap
import pyvista as pv
import numpy as np

# Visualization setup
p = pv.Plotter(notebook=True)

# Set the grid dimensions
grid = pv.ImageData()
grid.dimensions = np.array(final_agents.shape) + 1
# grid.origin과 grid.spacing 설정에 avail_lattice의 속성 사용
grid.origin = avail_lattice.minbound - avail_lattice.unit * 0.5
grid.spacing = avail_lattice.unit

# Adding the bounding box wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")
p.add_axes()
p.show_bounds(grid="back", location="back", color="#777777")

# Add the data values to the cell data
grid.cell_data["Agents"] = final_agents.flatten(order="F").astype(int)  # Flatten the array!

# Filtering the voxels
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

colors = [
    "#FFB6C1", "#FFC0CB", "#FF69B4", "#FF1493",  # 분홍색 계열
    "#87CEEB", "#00BFFF", "#1E90FF",  # 파란색 계열
    "#98FB98", "#00FF7F", "#32CD32", "#3CB371", "#2E8B57", "#228B22",  # 녹색 계열
    "#FFD700", "#FFA500", "#FF8C00", "#FF7F50", "#FF6347", "#FF4500", "#FF0000", "#DC143C", "#B22222",  # 노란색 계열
    "#BA55D3", "#9932CC", "#9400D3", "#8A2BE2", "#9370DB", "#7B68EE", "#6A5ACD",  # 자주색 계열
    "#FFE4B5", "#FFDAB9", "#FFDEAD",  # 베이지 계열
    "#F0E68C", "#EEE8AA", "#F5DEB3", "#DEB887", "#D2B48C", "#BC8F8F",  # 갈색 계열
    "#B0C4DE", "#ADD8E6", "#B0E0E6", "#AFEEEE", "#00CED1",  # 연한 파란색 계열
    "#696969",  # 회색 계열
    "#708090",  # 슬레이트 그레이 계열
    "#4682B4", "#5F9EA0", "#6495ED", "#7B68EE", "#87CEFA", "#87CEFA"  # 추가 파란색 계열
]

custom_cmap = ListedColormap(colors)
# Scalar bar 설정
sargs = dict(
    shadow=True,
    n_labels=agn_num,
    italic=False,
    fmt=" %.0f",
    font_family="arial",
    height=0.6,
    vertical=True,

    title_font_size=10,  # 제목 폰트 크기 설정
    label_font_size=6   # 라벨 폰트 크기 설정
)
annotations = {i: name for i, name in enumerate(space_list.values())}
# Adding the voxels with the custom colormap and scalar bar arguments
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)#

p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)

#envelope_lattice.fast_vis(p)
# Show the plot
p.show(jupyter_backend="trame", return_viewer=True)  # 시각화 표시


Widget(value='<iframe src="http://localhost:62579/index.html?ui=P_0x1553c412530_0&reconnect=auto" class="pyvis…

In [26]:
import numpy as np
import pyvista as pv
from matplotlib.colors import ListedColormap

# PyVista 플로터 초기화
p = pv.Plotter(notebook=True)

# 그리드 설정
grid = pv.ImageData()
grid.dimensions = np.array(final_agents.shape) + 1
grid.origin = avail_lattice.minbound - avail_lattice.unit * 0.5
grid.spacing = avail_lattice.unit

# 데이터 할당
grid.cell_data["Agents"] = final_agents.flatten(order="F").astype(int)

# 필터링
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

# 초기 컬러맵 설정
initial_colors = ["#808080" if i == 0 else "#FF0000" for i in range(len(program_prefs))]
custom_cmap = ListedColormap(initial_colors)

# 스칼라 바 설정
annotations = {i: name for i, name in enumerate(space_list.values())}
sargs = {
    "shadow": True,
    "n_labels": len(program_prefs),
    "italic": False,
    "fmt": " %.0f",
    "font_family": "arial",
    "height": 1,
    "vertical": True,
    "title_font_size": 10,
    "label_font_size": 6
}

# 메시 추가
opacity_map = np.full(threshed.n_cells, 0.1)  # 초기 투명도 설정
threshed.cell_data['opacity'] = opacity_map
mesh = p.add_mesh(threshed, scalars='opacity', cmap=custom_cmap, show_scalar_bar=True, scalar_bar_args=sargs, show_edges=True, opacity=1)

# 텍스트 추가
text_actor = p.add_text("Selected Program:", position='upper_right', font_size=12, color='black')
voxel_count_actor = p.add_text("Voxel Count: N/A", position='upper_left', font_size=12, color='black')

# 슬라이더 콜백 함수
def slider_callback(value):
    selected_program = int(value)
    mask = threshed.cell_data['Agents'] == selected_program
    opacity = np.full(threshed.n_cells, 0.1)  # 기본 투명도 설정
    opacity[mask] = 1.0  # 선택된 프로그램에 높은 투명도 부여

    # 투명도 업데이트
    threshed.cell_data['opacity'] = opacity
    
    # 텍스트 업데이트
    text_actor.SetText(0, f"Selected Program: {annotations[selected_program]}")
    
    # Voxel 개수 계산 및 표시
    current_voxels = np.sum(final_agents == selected_program)
    target_voxels = agent_areas[selected_program]
    difference = target_voxels - current_voxels
    
    voxel_count_actor.SetText(0, f"Current Voxels: {current_voxels}\nTarget Voxels: {target_voxels}\n")
    
    p.render()

# 슬라이더 위젯 추가
p.add_slider_widget(slider_callback, [0, len(program_prefs) - 1], value=0, title='Program Index')
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)

p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)

# 시각화 표시
p.show(jupyter_backend="trame", return_viewer=True)

Widget(value='<iframe src="http://localhost:62579/index.html?ui=P_0x15567e59840_10&reconnect=auto" class="pyvi…